# Char-based text generation with LSTM

In [33]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [34]:
TRAIN_TEXT_FILE_PATH = 'data.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'и', 'е', 'н', 'р', 'т', 'с', 'в', 'С', 'И', 'Р', 'О', 'А', 'л', 'к', '\n', 'Н', 'Е', 'п', 'д', 'К', 'у', 'Т', 'м', 'ы', 'В', 'з', 'М', 'я', 'Я', 'б', 'г', 'П', 'Л', 'й', 'Д', 'Б', '1', 'Г', 'ь', '0', 'З', 'ч', 'У', 'Ы', '2', 'ж', 'ц', 'х', 'Ц', '"', ',', 'Ф', 'ю', 'ш', 'ф', ':', 'Ш', 'Э', '3', '5', 'Ь', '9', 'D', 'Ж', '.', 'Ч', 'R', 'Й', 'O', 'I', '4', 'щ', 'A', 'Ю', 'э', '7', 'C', 'e', 'V', 'a', 'Х', '6', '8', 'n', '+', 'r', 'T', '%', 't', '/', 'E', '(', ')', 's', 'S', 'o', 'i', 'G', 'Щ', '$', 'c', 'M', 'N', 'Z', 'ъ', 'U', 'W', 'X', 'P', 'B', 'p', 'l', 'k', 'Ъ', 'd', 'J', 'w', 'F', 'f', 'z', 'h', 'u', '*', 'b', 'm', 'v', 'ё', '&', 'L', 'H', 'Y', 'K', ';', 'g', 'x', 'y', 'Q']


In [35]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [36]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [37]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [44]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.0703287172317504
 Сраде постостовит на 19 на постов в Моровити довния постов на рабали на простов прали на 19 на о повосстов остовна стодов остов на по в Модоля прастов пов пов постов на посков серания на посков о поли
Loss: 2.2747684001922606
  Моски в Моствертих на 13:00 МСК
 МИР ВАКЦИНА ПРОСТИКИ 11:00
 Резалинии на соблавении на сереновении в Моства в России в Моствании в Москов Московский остовном в Московский присли в РФ и США на 10 млн


KeyboardInterrupt: ignored

In [45]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

.  Моставил провых в странии странии "Фодольски в резовных продольской облении принитать по по по поления странии   Московской в РФ с в странии   Москов от потракцины острании   Мостарта по пристования поставии остовов акциями проводольской по сотратов РФ на по по пособлания проводольской по провых придет атала пристании с странии на постами в Москов в РФ притиков провых рестования простании проста на пристования простании в Предитать простова в РФ в РФ на простании по по по 11:01:01:01
 РОССИЯ ВАКЦИНА ПРОСС ВАКЦИНА ВАКЦИНА ПРОСС ПРЕДОНОВНОЙ РЫНОЗ ПРОСС ВАКЦИНА ПРОСС КРОСС ПРИГОСТАН ПРОЕНИЕ ПРОСС КРОСС РОССИЯ КРАСТА ПРОСС СТА ПРЕНИРЫ КРОСС ПРОСС КРОСС ИНДЕРОВОРС ПРОИЗВОСТАВИР ВАЗЫТИЯ ПРЕНИРЫ КОНОВНОЙ РЫНОЗ ПРОСС КРОСС КРОСС КУРСЫ
 Кросс проссии с пригание в Россия прогования продовния пристание проговать сатала   США по облистов на по проблинина по политать проиновном проводольской проставие продоблина по го посновных на поления прододоблания Astraeeaeca в по страстов в РФ раза США от 